In [1]:
import numpy as np
from numpy import linalg
# import test_mean.txt
test_means = np.loadtxt(open("test_mean.txt", "r"))
# hip1000.txt
expression = np.loadtxt(open("hip1000.txt", "r"), delimiter=",")
print(test_means.shape, expression.shape)
test_means = test_means[:10, ]
expression = expression[:10, :]
print(test_means.shape, expression.shape)
print((expression[0, :]).shape)
means = test_means
# 10x10
covariances = np.array(3*[np.identity(test_means.shape[0])])
# mixing proportions --prob_of_c
prob_of_c = np.array([0.3, 0.3, 0.4])
data = expression.copy()
print(prob_of_c.shape)
print(means)
print(covariances.shape)
print(data.shape)
print(data[:, 0].shape)

(208, 3) (208, 879)
(10, 3) (10, 879)
(879,)
(3,)
[[ 0.0765  0.001  -0.013 ]
 [-0.0906 -0.0305 -0.1173]
 [ 0.3749  0.2848  0.238 ]
 [ 0.2756  0.1082  0.1791]
 [ 0.2612  0.2003  0.1468]
 [ 0.0284  0.0293  0.0047]
 [-0.065   0.0675  0.0309]
 [ 0.0626  0.0351 -0.0085]
 [ 0.0233 -0.01    0.0247]
 [ 0.0104  0.0485 -0.103 ]]
(3, 10, 10)
(10, 879)
(10,)


In [2]:
def E_step(data, means, covariances, prob_of_c):
    # 10x879, 10x3, 3x10x10, 3x1
    hidden_matrix = [] 
    p_data_given_c = []
    
    for i in range(means.shape[1]):
        intermed_ls = []
        intermed_ls_2 = []
        for j in range(data.shape[1]):
            first_term = linalg.det(2*np.pi*covariances[i]) **(-1/2)
            data_minus_mean = (np.reshape((data[:,j]-means[:,i]), (1,10)) )
            second_term = (data_minus_mean @ linalg.inv(covariances[i]) @ data_minus_mean.T).item() * (-1/2)
            p_x_given_c_i = (first_term*np.exp(second_term))
            intermed_ls.append(p_x_given_c_i)
            intermed_ls_2.append(p_x_given_c_i*prob_of_c[i])
        p_data_given_c.append(intermed_ls)
        hidden_matrix.append(intermed_ls_2)
    # 3x879
    # 879x1
    p_data_given_c = np.array(p_data_given_c).T
    hidden_matrix = np.array(hidden_matrix)
    bottom = np.array([p_data_given_c[i,:] @ prob_of_c for i in range(p_data_given_c.shape[0])])
    for i in range(means.shape[1]):
        hidden_matrix[i] = hidden_matrix[i] / bottom
#     #print(p_data_given_c.shape, "should be 879x3")
#     # h matrix is wrong
#     hidden_matrix = [] #np.ones((879,3))
#     # need to correct bottom
    
    #print("p_data_given_c", p_data_given_c.shape)
    #print("prob_of_c", prob_of_c)
    #print("hidden_matrix", hidden_matrix.shape)
    #print(hidden_matrix.shape, "should be 879x3")
    return p_data_given_c,hidden_matrix.T

def get_log_likelihood(prob_of_c, prob_of_x_given_c):
#     prob_of_c = np.reshape(prob_of_c, (prob_of_c.shape[0],1))
    #print(prob_of_x_given_c.shape, prob_of_c.shape)
    log_likelihood = 0
    for i in range(prob_of_x_given_c.shape[0]):
        intermed_ls = 0
        for j in range(prob_of_c.shape[0]):
            intermed_ls += prob_of_x_given_c[i,j]*prob_of_c[j] 
        log_likelihood += np.log(intermed_ls)

    return log_likelihood

def M_step(data, hidden_matrix):
    N = data.shape[1]
#     print(N)
    # "should be 879x3", sum along the first axis
    # the hidden matrix is wrong, the probs should not sum to one immediately
    prob_of_c = np.sum(hidden_matrix, axis=0)/N
    # these should sum to 1
    #print(prob_of_c, "prob of c")
    # means 
    means = []
    for i in range(hidden_matrix.shape[1]):
        # check shape 
        hidden_matrix_i = np.reshape(hidden_matrix[:, i], (hidden_matrix.shape[0], 1))
        mean_i = (data @ hidden_matrix_i)/ np.sum(hidden_matrix[:, i], axis=0)
        means.append(mean_i)
    means = np.column_stack(means)
    #print(means.shape, "mean")
    # covariances
    covariances = []
    # 10x879 879x10 1x3 -- then does this across all data points 3
    # each cluster
    for i in range(hidden_matrix.shape[1]):
        # each datapoint
        intermediate_ls = []
        for j in range(hidden_matrix.shape[0]):
            datapoint = data[:,j] #10x879
            mean_i = means[:,i]
            data_minus_mean = datapoint - mean_i
            data_minus_mean = (np.reshape(data_minus_mean, (10, 1)))
            ele = data_minus_mean @ data_minus_mean.T * hidden_matrix[j, i] # 10x10
            intermediate_ls.append(ele)
        intermediate_ls = np.dstack(intermediate_ls)
        #print(np.sum(intermediate_ls, axis=2), "shloop")
        #print(np.sum(hidden_matrix[:, i], axis=0), "poop")
        covariances.append(np.sum(intermediate_ls, axis=2) / np.sum(hidden_matrix[:, i], axis=0))
    covariances = np.array(covariances) 
    #print(covariances.shape, "covariance")
    return prob_of_c, means, covariances
        
p_data_given_c, hidden_matrix = E_step(data, means, covariances, prob_of_c)    
#print(get_log_likelihood(prob_of_c, p_data_given_c))
#print(hidden_matrix, "HIDDEN")
prob_of_c_1, means_1, covariances_1 = M_step(data, hidden_matrix)

In [3]:
def E_M_algo(prob_of_c, means, covariances, data, conv_thresh):
    log_likelihoods = []
    i = 0
    # now start the while loop
    while (i < 3) or ((log_likelihoods[i-1] - log_likelihoods[i-2]) > conv_thresh):
        # not using the new params!
        p_data_given_c, hidden_matrix = E_step(data, means, covariances, prob_of_c)
        prob_of_c, means, covariances = M_step(data, hidden_matrix)
        #print("hidden", hidden_matrix[:,0])
        #print("covariances", covariances[0,:,:])
        #print(prob_of_c)
        log_likelihood = get_log_likelihood(prob_of_c, p_data_given_c)
        log_likelihoods.append(log_likelihood)
        i += 1
    return(log_likelihoods, hidden_matrix, p_data_given_c)
    
conv_thresh = 0.001
log_likelihoods, hidden_matrix, p_data_given_c = E_M_algo(prob_of_c, means, covariances, data, conv_thresh)
import matplotlib.pyplot as plt
plt.plot([i for i in range(len(log_likelihoods))], log_likelihoods)
plt.title("Log Likelihood for K=3 with given initial centers")
plt.xlabel("number of iterations")
plt.ylabel("Log Likelihood")
plt.savefig("3a.png")
plt.close()

In [4]:
print(hidden_matrix[0].shape)
N = data.shape[1]
prob_of_c = hidden_matrix[0]
print(prob_of_c)
print(np.sum(prob_of_c))

(3,)
[3.85944833e-05 1.25086262e-01 8.74875144e-01]
1.0


In [6]:
#For K = 3, try 10 different random initializations for all parameters
def random_inits(data, k):
    # prob of c -- 3x1 -- must sum to 1
    r = [np.random.random() for i in range(k)]
    s = sum(r)
    r = np.array([ i/s for i in r ])
    prob_of_c = r
    #print((prob_of_c))
    #print(sum(((prob_of_c))))
    # mean -- 10x3
    means = np.random.rand(10, k)
    #print(means)
    # covariance -- 3x10x10
    identities = np.array(k*[np.identity(test_means.shape[0])])
    epsilon = 0.001
    covariances = []
    for i in range(k):
        covariance = (np.random.rand((10))*identities[i]) + epsilon
        covariances.append(covariance)
    covariances = np.dstack(covariances).T
    
    return prob_of_c, means, covariances

#  must do this 10 times for this one
def sim_for_k(number_of_simulations, data, k):
    out = []
    np.seterr(over='raise')
    for i in range(number_of_simulations):
        while True:
            try:
                prob_of_c, means, covariances = random_inits(data, k)
                log_likelihoods, hidden_matrix, p_data_given_c = E_M_algo(prob_of_c, means, covariances, data, conv_thresh)        
                print(log_likelihoods[-1])
                break
            except:
                pass
        out.append(log_likelihoods[-1])
    print(out)

sim_for_k(10, data, 3)
# What is the data log-likelihood at convergence for each initialization?
# [Hint: There should be 10 values]. Which one do you think was the best initialization among the 10?

-10340.527106915102
-10307.64187787189
-10334.177243815871


/var/folders/86/ytzggnn50y3gt2z3d8dh6bm80000gn/T/ipykernel_2814/2053794297.py:10: RuntimeWarning: invalid value encountered in double_scalars
  first_term = linalg.det(2*np.pi*covariances[i]) **(-1/2)


-10317.775438424296
-10323.230613067317
-10317.501010040458
-10360.793964925024
-10316.021247120898
-10526.337337187588
-10564.175683938727
[-10340.527106915102, -10307.64187787189, -10334.177243815871, -10317.775438424296, -10323.230613067317, -10317.501010040458, -10360.793964925024, -10316.021247120898, -10526.337337187588, -10564.175683938727]


In [8]:
def simulate_me(number_of_simulations, data):
    # init the split in the beginning -- 10, 879
    train, test = data[:,:600], data[:,600:]
    ll_ls = []
    # obviously this was not worth it-- you just got unlucky
    np.seterr(all='raise')
    for k in range(3, 11):
        test_ls = []
        for i in range(number_of_simulations):
            while True:
                try:
                    prob_of_c, means, covariances = random_inits(data, k)
                    log_likelihoods, hidden_matrix, p_data_given_c = E_M_algo(prob_of_c, means, covariances, data, conv_thresh)        
                    print(log_likelihoods[-1])
                    break
                except:
                    pass
            # eval on test
            N = train.shape[1]
            #print(hidden_matrix.shape, N)
            prob_of_c = np.sum(hidden_matrix, axis=0)/N
            p_data_given_c, hidden_matrix = E_step(test, means, covariances, prob_of_c)
            test_ls.append(get_log_likelihood(prob_of_c, p_data_given_c))
        #print(test_ls)
        keep_ll = np.max(test_ls)
        print(keep_ll)
        ll_ls.append(keep_ll)
        
    plt.plot([i+3 for i in range(len(ll_ls))], ll_ls)
    plt.title("Log Likelihood for K from 3 to 10 with random initial centers")
    plt.xlabel("K")
    plt.ylabel("Best Test Log Likelihood")
    #plt.show()
    plt.savefig("3d.png")
    plt.close()
            
            
                
number_of_simulations = 10         
simulate_me(number_of_simulations, data)  

-10331.905353771952
-10316.255578693012
-10545.764027870982
-10318.452892839241
-10324.657959419206
-10304.48330494598
-10326.651483792553
-10305.283894324604
-10520.658438596858
-10332.024809210769
-4844.957359364966
-10224.606917285313
-10251.366051790452
-10222.022187425153
-10221.378910112242
-10228.603673647407
-10249.958691039565
-10247.806343740773
-10226.924003008171
-10224.354728761333
-10213.956607690941
-4196.19890241876
-10140.287474859566
-10204.255814379285
-10154.031085207425
-10181.39830009955
-10183.143026443156
-10173.350386834909
-10174.51082856208
-10176.000176043197
-10170.677897900887
-10166.088548960272
-4663.086072617946
-10079.795944571903
-10108.774704591038
-10064.607894485982
-10086.32381149467
-10114.556932040834
-10124.737863447668
-10079.568326308892
-10056.831829545685
-10104.2750789858
-10067.690783351454
-4611.132910676744
-9993.69075999873
-10010.393066162847
-10028.000522947004
-10050.86210502862
-10028.724734581705
-9997.338409015561
-10029.83496848

In [ ]:
 # Compute the likelihood of the held-out (validation) data to determine which model (number of clusters) is more accurate